## 전치행렬과 max 값을 이용하여 데이터 Normalize 한 후,
## training data 와 test data 분리

In [1]:
import numpy as np
import random
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# Wine Class

class Wine:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, name, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        self.name = name
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_nodes)  
        self.b2 = np.random.rand(hidden_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_nodes,output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print(self.name, " is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        # list which contains (index, label, prediction) value
        index_label_prediction_list = []
        
        # temp list which contains label and prediction in sequence
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
                temp_list.append(index)
                temp_list.append(target_data[index])
                temp_list.append(logical_val)
                
                index_label_prediction_list.append(temp_list)
                
                temp_list = []
                
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

## wine.csv 를 읽어 온 후에,
## 각 열의 최대값을 구하기 위해 원본 행렬을 전치행렬로 변환함

In [3]:
# Normalize 않되어 있는 ThoracicSurgery.csv 읽어옴
loaded_data = np.loadtxt('./wine.csv', delimiter=',', dtype=np.float32)

print("loaded_data.shape = ", loaded_data.shape)

# 각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치향렬을 만들어줌
transpose_loaded_data = loaded_data.T

print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)

loaded_data.shape =  (6497, 13)
transpose_loaded_data.shape =  (13, 6497)


## 전치된 행렬에서 각 행의 최대 값을 찾아 나누어 주면서 Normalize 시킨 후에,
## 다시 전치행렬을 만들어서 csv 파일로 저장함

In [4]:
# 전치행렬을 위한 리스트
transpose_normalize_data_list = []

for index in range(len(transpose_loaded_data)):
    
    max_value = np.max(transpose_loaded_data[index, :])   # 각 행의 최대값을 찾음
    
    # 최대값이 1 이상이면 최대값으로 나누어서 
    # 모든 데이터가 0 ~ 1 사이에 오도록 함
    if max_value > 1.0:  
        
        transpose_normalize_data_list.append(transpose_loaded_data[index, :] / max_value)
       
    # 최대값이 1 보다 작으면 해당 값을 그대로 사용함. 
    ## 왜냐하면 1보다 작은 값이면 굳이 바꿀 필요가 없음
    else:      
        
        transpose_normalize_data_list.append(transpose_loaded_data[index, :])
        

# 리스트를 numpy type 으로 변환
transpose_normalize_data = np.array(transpose_normalize_data_list)

print(transpose_normalize_data.shape)

# 데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
normalize_data = transpose_normalize_data.T

print(normalize_data.shape)

# normalize 파일저장
np.savetxt('./Normalize_Wine_data.csv', normalize_data, delimiter=',')

(13, 6497)
(6497, 13)


In [5]:
# 데이터 불러오기

loaded_data = np.loadtxt('./Normalize_Wine_data.csv', delimiter=',', dtype=np.float32)

print("loaded_data.shape = ", loaded_data.shape)

# 임시 저장 리스트
training_data_list = []
test_data_list = []

# 전체의 40%를 테스트데이터로 분리
total_data_num = len(loaded_data)
seperation_rate = 0.4
test_data_num = int(len(loaded_data) * seperation_rate)

print("total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

# 전체 데이터 인덱스를 가지고 있는 리스트 생성
total_data_index_list = [ index for index in range(total_data_num) ]

# random.sample 을 이용하여 테스트 데이터를 위한 인덱스 리스트 생성
test_data_index_list = random.sample(range(total_data_num), test_data_num)

print("length of test_data_index_list = ", len(test_data_index_list))


# training data 는 전체 데이터에서 test_data 를 위한 index 를 제외하면 됨
for index in range(len(test_data_index_list)):
    
    total_data_index_list.remove(test_data_index_list[index])
    

# training data 구성
for training_data_index in total_data_index_list:
    
    training_data_list.append(loaded_data[training_data_index])

# test data 구성
for test_data_index in test_data_index_list:
    
    test_data_list.append(loaded_data[test_data_index])

# generate training data from training_data_list using np.arrya(...)
training_data = np.array(training_data_list)

# generate test data from test_data_list using np.arrya(...)
test_data = np.array(test_data_list)

# verification shape
print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

# save training & test data (.csv)
np.savetxt('./random_wine_normalized_training_data.csv', training_data, delimiter=',')
np.savetxt('./random_wine_normalized_test_data.csv', test_data, delimiter=',')

loaded_data.shape =  (6497, 13)
total_data_num =  6497 , test_data_num =  2598
length of test_data_index_list =  2598
training_data.shape =  (3899, 13)
test_data.shape =  (2598, 13)


In [15]:
#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 5  # hidden nodes 개수. 
o_nodes = 1    # output nodes 개수
lr = 1e-4      # learning rate. hi_node = 1, 1e-1, 1e-2 에서는 발산
epochs = 20   # 반복횟수. 

# Wine 객체 생성
obj = Wine("WINE", i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

WINE  is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  2.334120973349792
epochs =  1 loss value =  1.2862780806935439
epochs =  2 loss value =  0.7377022868902284
epochs =  3 loss value =  0.4964511011799369
epochs =  4 loss value =  0.3847193163557317
epochs =  5 loss value =  0.32707661809195954
epochs =  6 loss value =  0.29462388358582425
epochs =  7 loss value =  0.27521882852835516
epochs =  8 loss value =  0.26314109204380104
epochs =  9 loss value =  0.2554216339533987
epochs =  10 loss value =  0.2503999321992277
epochs =  11 loss value =  0.24709444903403582
epochs =  12 loss value =  0.24490132313067067
epochs =  13 loss value =  0.24343835869409727
epochs =  14 loss value =  0.24245881123124885
epochs =  15 loss value =  0.24180118296610997
epochs =  16 loss value =  0.24135877152261803
epochs =  17 loss value =  0.2410606237566577
epochs =  18 loss value =  0.24085935197572905
epochs =  19 loss value =  0.24072321340892824

Ela

In [16]:
test_data = np.loadtxt('./random_wine_normalized_test_data.csv', delimiter=',', dtype=np.float32)
print("test_data.shape = ", test_data.shape)

test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list_1, false_list_1, index_label_prediction_list) = obj.accuracy(test_input_data, test_target_data) 

test_data.shape =  (2598, 13)
Accuracy =>  0.7548113933795227


In [17]:
print(index_label_prediction_list)

[[0, 1.0, 0], [1, 1.0, 0], [5, 1.0, 0], [8, 1.0, 0], [20, 1.0, 0], [25, 1.0, 0], [28, 1.0, 0], [34, 1.0, 0], [38, 1.0, 0], [39, 1.0, 0], [40, 1.0, 0], [41, 1.0, 0], [43, 1.0, 0], [48, 1.0, 0], [54, 1.0, 0], [58, 1.0, 0], [59, 1.0, 0], [61, 1.0, 0], [65, 1.0, 0], [66, 1.0, 0], [68, 1.0, 0], [72, 1.0, 0], [73, 1.0, 0], [74, 1.0, 0], [79, 1.0, 0], [83, 1.0, 0], [91, 1.0, 0], [95, 1.0, 0], [96, 1.0, 0], [97, 1.0, 0], [105, 1.0, 0], [106, 1.0, 0], [107, 1.0, 0], [116, 1.0, 0], [119, 1.0, 0], [123, 1.0, 0], [126, 1.0, 0], [131, 1.0, 0], [132, 1.0, 0], [141, 1.0, 0], [143, 1.0, 0], [155, 1.0, 0], [157, 1.0, 0], [164, 1.0, 0], [166, 1.0, 0], [175, 1.0, 0], [177, 1.0, 0], [178, 1.0, 0], [180, 1.0, 0], [187, 1.0, 0], [188, 1.0, 0], [191, 1.0, 0], [205, 1.0, 0], [211, 1.0, 0], [219, 1.0, 0], [220, 1.0, 0], [223, 1.0, 0], [227, 1.0, 0], [229, 1.0, 0], [240, 1.0, 0], [241, 1.0, 0], [242, 1.0, 0], [243, 1.0, 0], [244, 1.0, 0], [251, 1.0, 0], [254, 1.0, 0], [255, 1.0, 0], [256, 1.0, 0], [258, 1.0, 0]